<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepare-functions" data-toc-modified-id="Prepare-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare functions</a></span></li><li><span><a href="#WT-model" data-toc-modified-id="WT-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>WT model</a></span></li><li><span><a href="#BRAF-mutated-model" data-toc-modified-id="BRAF-mutated-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>BRAF mutated model</a></span></li><li><span><a href="#RAS-mutation" data-toc-modified-id="RAS-mutation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>RAS mutation</a></span></li><li><span><a href="#EGFR-mutation" data-toc-modified-id="EGFR-mutation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>EGFR mutation</a></span></li></ul></div>

# Calculate the true local response coefficients and direct perturbation effects

In this notebook, I calculate the true local response coefficients and direct perturbation effects of the Orton model and "mutated" derivatives.

The local response coefficients are calculated as follows:
  * Calculate model steady state
  * Numerically calculate the derivative of a module by changing its imput modules by a small delta and recalculating the steady state of that module only. Other inputs are fixed to their steady state value.
  * rloc is defined as (xii/xoi)*(xof-xoi)/(xif-xii), where:
    
    * xi = input node
    * xo = output node
    * xxi = initial level
    * xxo = final level
 
The perturbations effects are calculated as follows:
  * Calculate model steady state.
  * Reduce initial amound of node (NodeInactive) by 50%
  * Recalculate steady state with input nodes fixed to original steady state.
  * rpert is defined as: 2*(xof - xoi)/(xoi+xof))
  
These are all chosed to match the simulations [Model simulations](model simulation.ipynb)

Of note: Here we calculate steady state only by time-course integration.
Solving the steady state equations using root-finding algorithms is unstable because moeities are not explicitly concerved in these.


In [3]:
import os
import sys

os.chdir("/Users/e.bosdriesz/Dropbox/projects/cnr/orton/")

import pickle
import numpy as np
import odemod
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import moduleForOdeModeling
import scipy.integrate
import scipy.optimize
import sympy
import copy


%matplotlib inline
np.set_printoptions(precision=3, suppress=1, linewidth=150)

###############################################################################
# INPUTS
fModelDict = 'ortonModelExtendedDictionary.pkl'


###############################################################################
# STATEMENTS

# Importing the model
with open(fModelDict, 'rb') as handle:
    mDict = pickle.load(handle, encoding='latin1')


## Prepare functions

In [4]:
simWT = odemod.SimPerturbations(mDict)

nodes = set()
for module in simWT.modules.values():
    nodes.update(module["input_from"])

# Remove negative regulators without input from nodes
not_nodes = set()
for node in nodes:
    if simWT.get_species_names(node) in ["Rap1Gap", 'PP2AActive', 'RasGapActive', "Raf1PPtase"]:
        not_nodes.add(node)
nodes = nodes - not_nodes

NODES_ORDERED = [simWT.get_species_names(sp) for sp in simWT.idSp if sp in nodes]

In [5]:
def get_responses_for_module(m, module_name):
    
    # Data for all responses
    module = m.modules[module_name]
    # Reactions, species and rate equations this module
    rxns = list(module["reactions"])
    sps = list(module["species"])
    rates = {rx: m.rateDict[rx] for rx in rxns}
    # Get the stochiometry submatrix: module species x module reaction
    spIdx = np.array([m.idSp.index(sp) for sp in sps]) # indices of species
    rnIdx = np.array([m.idRs.index(rx) for rx in rxns]) # indices of reactions
    S_module = m.S[spIdx, :][: , rnIdx]

    # Output species of the module (can be only 1)
    output_node = [sp for sp in sps if sp in nodes]
    assert len(output_node) == 1
    output_node = output_node[0]
    #input nodes to the module
    input_nodes = [n for n in module['input_from'] if n in nodes]
    
    # Parameters related to the module
    # Treat input nodes as parameter. 
    # These will be perturbed to get local response coefficient
    param = dict()
    for sp in module['input_from']:
        param[sp] = dict(m.refSS)[sp]
    for p in module['param']:
        param[p] = m.param[p]
    
    ss_module = [dict(m.refSS)[sp] for sp in sps]

    # Calculate responses
    response_lst = []
    for node_perturbed in input_nodes:
        param_perturbed = copy.deepcopy(param)
        param_perturbed[node_perturbed] = param_perturbed[node_perturbed]*1.001

        v_perturbed = moduleForOdeModeling.createRateVector(
            sps, rxns, rates, param_perturbed
        )
        def odes_perturbed(x, t):
            return np.dot(S_module, v_perturbed(x))   
        simTime = np.arange(0, 500, 0.01)
        tsol = dict(zip(sps, scipy.integrate.odeint(odes_perturbed, ss_module, simTime)[-1]))
        xof, xoi = tsol[output_node], dict(m.refSS)[output_node]
        xif, xii = param_perturbed[node_perturbed], dict(m.refSS)[node_perturbed]

        response_scaled = (xii/xoi)*(xof-xoi)/(xif-xii)
        
        response_lst.append((
            m.get_species_names(output_node),
            m.get_species_names(node_perturbed),
            response_scaled
            )
        )
        
    return response_lst

def build_rloc(m):
    # NODES_ORDERED = [m.get_species_names(sp) for sp in m.idSp if sp in nodes]
    rloc = -1*np.identity(len(nodes))
    for module in m.modules.keys():
        response_lst = get_responses_for_module(m, module)
        for response in response_lst:
            i = NODES_ORDERED.index(response[0])
            j = NODES_ORDERED.index(response[1])
            rloc[i, j] = response[2]
    rloc = pd.DataFrame(rloc)
    rloc.index = NODES_ORDERED
    rloc.columns = NODES_ORDERED
    return rloc

In [6]:
def get_direct_kd_effect(m, species_kd):
    module = []
    for key, value in m.modules.items():
        if species_kd in value['species']:
            module.append(value)
    assert len(module) == 1
    module = module[0]

    rxns = list(module["reactions"])
    sps = list(module["species"])
    rates = {rx: m.rateDict[rx] for rx in rxns}
    # Get the stochiometry submatrix: module species x module reaction
    spIdx = np.array([m.idSp.index(sp) for sp in sps]) # indices of species
    rnIdx = np.array([m.idRs.index(rx) for rx in rxns]) # indices of reactions
    S_module = m.S[spIdx, :][: , rnIdx]

    # Output species of the module (can be only 1)
    output_node = [sp for sp in sps if sp in nodes]
    assert len(output_node) == 1
    output_node = output_node[0]
    
    param_kd = dict()
    for sp in module['input_from']:
        param_kd[sp] = dict(m.refSS)[sp]
    for p in module['param']:
        param_kd[p] = m.param[p]

    init = []
    for sp in sps:
        ival = dict(zip(m.idSp, m.init))[sp]
        if sp == species_kd:
            init.append(0.5*ival)
        else:
            init.append(ival)

    v_kd = moduleForOdeModeling.createRateVector(
        sps, rxns, rates, param_kd
    )        

    def odes_kd(x, t):
        return np.dot(S_module, v_kd(x))   

    simTime = np.arange(0, 1e4, 1)
    tsol = dict(zip(sps, scipy.integrate.odeint(odes_kd, init, simTime)[-1]))
    xof, xoi = tsol[output_node], dict(m.refSS)[output_node]
    return (m.get_species_names(output_node), 2*(xof - xoi)/(xoi+xof))

def build_rpert(m, knockdown):
    rp = np.zeros(shape = (len(nodes), len(knockdown)))
    targets = []
    j = 0
    for kd in knockdown:
        # List of nodes that are directly perturbed
        kd_nodes = []
        # NODES_ORDERED = [m.get_species_names(n) for n in m.idSp if n in nodes]
        kd_response = get_direct_kd_effect(m, kd)
        targets.append(kd_response[0]+"Knockdown")
        i = NODES_ORDERED.index(kd_response[0])
        rp[i][j] = kd_response[1]
        j += 1
    rp = pd.DataFrame(rp, index = NODES_ORDERED, columns = targets)
    return rp

## WT model

In [7]:

knockdown_wt = [
    'species_3', # Module 1: SOS
    'species_5',  # Module 2: Ras
    'species_7',  # Module 3: Raf
    'species_9',  # Module 4: Mek
    'species_11', # Module 5: Erk
    'species_13', # Module 6: P90Rsk
    'species_15',  # Module 7: PI3K
    'species_17',  # Module 8: Akt
    'species_20',  # Module 9: C3G
    'species_22',  # Module 10: Rap1
    'species_24'   # Module 11: bRaf
]
rloc_wt = build_rloc(simWT)

rpert_wt = build_rpert(simWT, knockdown_wt)
# We need to add EGFR KD seperately because it is not simulated by reducing inial amoount by 50%
cols = list(rpert_wt.columns)
# Module 0: EGFR. Model perturbation by reducing production rate
simWT.addPerturbation('v', 0.5 * simWT.param['v'])
rpert_wt["boundEGFReceptorKnockdown"] = pd.Series([simWT.rglob[0, 0]] + [0]*(len(nodes)-1), index=NODES_ORDERED)
rpert_wt = rpert_wt[["boundEGFReceptorKnockdown"] + cols]


rloc_wt.to_csv('results/simulations/ortonModel-knockdown-mutWT-rloc.tsv', sep='\t')
rpert_wt.to_csv('results/simulations/ortonModel-knockdown-mutWT-rpert.tsv', sep='\t')


## BRAF mutated model

In [8]:
simBRAF = odemod.SimPerturbations(mDict)
# No (de)-activation of BRaf
# Kcat_25 = Activation by Rap1
# Kcat_26 = Activation by Ras
# Kcat_30 = deactivation by p'ase
simBRAF.setParam(['Kcat_25', 'Kcat_26', 'Kcat_30'], [0, 0, 0])
# BRafInactive absent. All Braf is active.
simBRAF.setInit(['species_23', 'species_24'], [120000, 0])

simBRAF.update_refSS()

knockdown_braf = [
    'species_3', # Module 1: SOS
    'species_5',  # Module 2: Ras
    'species_7',  # Module 3: Raf
    'species_9',  # Module 4: Mek
    'species_11', # Module 5: Erk
    'species_13', # Module 6: P90Rsk
    'species_15',  # Module 7: PI3K
    'species_17',  # Module 8: Akt
    'species_20',  # Module 9: C3G
    'species_22',  # Module 10: Rap1
    # 'species_24'   # Module 11: bRaf
    'species_23' # Module 11: BRAF (use active BRAF instead of inactive BRAF).
]


rloc_braf = build_rloc(simBRAF)

rpert_braf = build_rpert(simBRAF, knockdown_braf)
# We need to add EGFR KD seperately because it is not simulated by reducing inial amoount by 50%
cols = list(rpert_braf.columns)
# Module 0: EGFR. Model perturbation by reducing production rate
simBRAF.addPerturbation('v', 0.5 * simBRAF.param['v'])
rpert_braf["boundEGFReceptorKnockdown"] = pd.Series([simBRAF.rglob[0, 0]] + [0]*(len(nodes)-1), index=NODES_ORDERED)
rpert_braf = rpert_braf[["boundEGFReceptorKnockdown"] + cols]



rloc_braf.to_csv('results/simulations/ortonModel-knockdown-mutBRAF-rloc.tsv', sep='\t')
rpert_braf.to_csv('results/simulations/ortonModel-knockdown-mutBRAF-rpert.tsv', sep='\t')



## RAS mutation

In [12]:

simRas = odemod.SimPerturbations(mDict)
# No (de)-activation of Ras
simRas.setParam(['Kcat_03', 'Kcat_04'], [0, 0])
# RasInactive absent. All Ras is active.
simRas.setInit(['species_4', 'species_5'],[120000, 0])

simRas.update_refSS()

knockdown_ras = [
    'species_3', # Module 1: SOS
    # 'species_5',  # Module 2: Ras (use active RAS instead of inactive RAS).
    'species_4',   # Active Ras
    'species_7',  # Module 3: Raf 
    'species_9',  # Module 4: Mek
    'species_11', # Module 5: Erk
    'species_13', # Module 6: P90Rsk
    'species_15',  # Module 7: PI3K
    'species_17',  # Module 8: Akt
    'species_20',  # Module 9: C3G
    'species_22',  # Module 10: Rap1
    'species_24',   # Module 11: bRaf
]


rloc_ras = build_rloc(simRas)

rpert_ras = build_rpert(simRas, knockdown_ras)
# We need to add EGFR KD seperately because it is not simulated by reducing inial amoount by 50%
cols = list(rpert_ras.columns)
# Module 0: EGFR. Model perturbation by reducing production rate
simRas.addPerturbation('v', 0.5 * simRas.param['v'])
rpert_ras["boundEGFReceptorKnockdown"] = pd.Series([simRas.rglob[0, 0]] + [0]*(len(nodes)-1), index=NODES_ORDERED)
rpert_ras = rpert_ras[["boundEGFReceptorKnockdown"] + cols]



rloc_ras.to_csv('results/simulations/ortonModel-knockdown-mutRas-rloc.tsv', sep='\t')
rpert_ras.to_csv('results/simulations/ortonModel-knockdown-mutRas-rpert.tsv', sep='\t')



## EGFR mutation

In [11]:
simEGFR = odemod.SimPerturbations(mDict)
# No (de)-activation of EGFR
# * k1_00: EGF-EGFR-free binding
# * k2_00: EGF-EGFR unbiding
# * k1_20: EGFR degradation
# * k1_29: free EGFR degradation
# * v:     EGFR production
simEGFR.setParam(['k1_00', 'k2_00', 'k1_20', 'k1_29', 'v'], [0, 0, 0, 0, 0])
# All EGFR is Active.
simEGFR.setInit(['species_0', 'species_1'], [80000, 0])

simEGFR.update_refSS()


knockdown_egfr = [
    # Now, add EGFR as knockdown
    'species_0', 
    'species_3', # Module 1: SOS
    'species_5',  # Module 2: Ras
    'species_7',  # Module 3: Raf 
    'species_9',  # Module 4: Mek
    'species_11', # Module 5: Erk
    'species_13', # Module 6: P90Rsk
    'species_15',  # Module 7: PI3K
    'species_17',  # Module 8: Akt
    'species_20',  # Module 9: C3G
    'species_22',  # Module 10: Rap1
    'species_24'   # Module 11: bRaf
]


rloc_egfr = build_rloc(simEGFR)

rpert_egfr = build_rpert(simEGFR, knockdown_egfr)
# We need to add EGFR KD seperately because it is not simulated by reducing inial amoount by 50%
cols = list(rpert_egfr.columns)
# Module 0: EGFR. Model perturbation by reducing production rate
# simEGFR.addPerturbation('v', 0.5 * simEGFR.param['v'])
# rpert_egfr["boundEGFReceptorKnockdown"] = pd.Series([simEGFR.rglob[0, 0]] + [0]*(len(nodes)-1), index=NODES_ORDERED)
# rpert_egfr = rpert_egfr[["boundEGFReceptorKnockdown"] + cols]





In [13]:
rloc_egfr.to_csv('results/simulations/ortonModel-knockdown-mutEGFR-rloc.tsv', sep='\t')
rpert_egfr.to_csv('results/simulations/ortonModel-knockdown-mutEGFR-rpert.tsv', sep='\t')


,boundEGFReceptorKnockdown,SosActiveKnockdown,RasActiveKnockdown,Raf1ActiveKnockdown,MekActiveKnockdown,ErkActiveKnockdown,P90RskActiveKnockdown,PI3KActiveKnockdown,AktActiveKnockdown,C3GActiveKnockdown,Rap1ActiveKnockdown,bRafActiveKnockdown
boundEGFReceptor,-0.666667,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
SosActive,0.000000,-0.661956,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
RasActive,0.000000,0.000000,-0.21098,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
Raf1Active,0.000000,0.000000,0.00000,-0.296692,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
MekActive,0.000000,0.000000,0.00000,0.000000,-0.692473,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
ErkActive,0.000000,0.000000,0.00000,0.000000,0.000000,-0.664741,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000
P90RskActive,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,-0.660806,0.000000,0.000000,0.00000,0.000000,0.00000
PI3KActive,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-0.666666,0.000000,0.00000,0.000000,0.00000
AktActive,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.657635,0.00000,0.000000,0.00000
C3GActive,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.66626,0.000000,0.00000
